In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

class BinaryClassificationDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

class Transformer(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout=0.1):
        super(Transformer, self).__init__()
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward,
                                          dropout=dropout)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, src, tgt):
        memory = self.transformer.encoder(src)
        output = self.transformer.decoder(tgt, memory)
        output = self.fc(output)
        return output.squeeze()

def train(model, train_loader, optimizer, criterion):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.transpose(0, 1), target[:-1].unsqueeze(0))
        loss = criterion(output, target[1:])
        loss.backward()
        optimizer.step()

def evaluate(model, val_loader):
    model.eval()
    total_loss = 0.
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            output = model(data.transpose(0, 1), target[:-1].unsqueeze(0))
            loss = criterion(output, target[1:])
            total_loss += loss.item()
    return total_loss / len(val_loader)

# Example usage
train_data = torch.randn(1000, 10)
train_targets = torch.randint(0, 2, (1000,))
train_dataset = BinaryClassificationDataset(train_data, train_targets)
train_loader = DataLoader(train_dataset, batch_size=32)

val_data = torch.randn(1000, 10)
val_targets = torch.randint(0, 2, (1000,))
val_dataset = BinaryClassificationDataset(val_data, val_targets)
val_loader = DataLoader(val_dataset)

model = Transformer(d_model=10,
                     nhead=2,
                     num_encoder_layers=2,
                     num_decoder_layers=2,
                     dim_feedforward=32)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(10):
    train(model=model,
          train_loader=train_loader,
          optimizer=optimizer,
          criterion=criterion)
    val_loss = evaluate(model=model,
                        val_loader=val_loader)
    print(f"Epoch {epoch}, Val Loss: {val_loss:.4f}")


AssertionError: was expecting embedding dimension of 10, but got 32